In [25]:
import pandas as pd
import numpy as np
from datetime import date #, datetime, timedelta

# import time
# import itertools

In [ ]:
# !!!

# from ... import to_split_and_scale, namestr

<a id="1"></a>

# 1. Downloading data

In [6]:
price_df4 = pd.read_csv("poland_apartments_completed.csv")
price_df4.head()

,Unnamed: 0,id,city,floor,latitude,longitude,price,rooms,sq,year,latitude_center,longitude_center,decade,price_per_m,sq_round,radius,radius_dec,radius_ceil,district
0,0,23918.0,Kraków,2,50.049224,19.970379,749000,3,74,2021,50.061389,19.937222,2020,10121,70,2.732975,2.7,3,Dzielnica XIII Podgórze
1,1,17828.0,Warszawa,3,52.249775,21.106886,240548,1,24,2021,52.230000,21.011111,2020,10022,20,6.902500,6.9,7,Praga Południe
2,2,22784.0,Kraków,2,50.066964,19.920025,427000,2,37,1970,50.061389,19.937222,1970,11540,30,1.378660,1.4,2,Dzielnica V Krowodrza
3,3,4315.0,Poznañ,2,52.404212,16.882542,1290000,5,166,1935,52.408333,16.933611,1930,7771,160,3.505600,3.5,4,Grunwald
4,4,11770.0,Warszawa,1,52.212225,20.972630,996000,5,105,2020,52.230000,21.011111,2020,9485,100,3.290526,3.3,4,Ochota


In [7]:
cat_features = ['city', 'district'] # 'floor', 'rooms'?
num_features = ['floor', 'rooms', 'sq', 'year', 'radius']
target = ['price']


In [ ]:
# WHAT NEEDS FOR LAUNCHING:

# price_df4 # +
# cat_features # +
# labels_dict # ???? from to_split_and_scale() ?
# st_scaler # from to_split_and_scale()
# reg_xgb # best model

<a id="5"></a>

# 5. Predicting a price by example features


In [29]:
def check_city_district_radius_floor_rooms(city, district, radius, floor, rooms):
    
    price_df4 = pd.read_csv("poland_apartments_completed.csv")
    floor_values = list(set(price_df4['floor']))
    rooms_values = list(set(price_df4['rooms']))
    
    geo_df_for_check = price_df4.groupby(['city', 'district']).agg({'radius':['min','max']})
    
    if (city, district) not in geo_df_for_check.index:
        print("Invalid city and/or district!")
        return False
    else:
        min_radius = geo_df_for_check.loc[(city, district)][('radius', 'min')]
        max_radius = geo_df_for_check.loc[(city, district)][('radius', 'max')]
        if not (radius >= min_radius and radius <= max_radius):
            print("The radius must be between {0} and {1}".format(round(min_radius, 2), round(max_radius, 2)))
            return False
        elif floor not in floor_values:
            print("The floor must be integer and between {0} and {1}".format(floor_values.min(), floor_values.max()))
            return False
        elif rooms not in rooms_values:
            print("The rooms must be integer and between {0} and {1}".format(rooms_values.min(), rooms_values.max()))
            return False
        else:
            return True
#         else:    
#             for col in [floor, rooms]: # city, district, 
#                 if col not in list(set(price_df4[namestr(col, globals())[0]])):
#                     print(f"Invalid {col}")
#                     return False
    

In [33]:
def check_sq(sq):
    
    if (
        type(sq) in [int, float] and 
        sq >= 20 and
        sq <= 100
    ):
        return True
    else:
        print("The sq should be between {0} and {1}".format(20, 100))
        return False


In [37]:
def check_year(year, city):
    
    city_foundations = {
        'Warszawa': 1300,
        'Kraków': 990,
        'Poznañ': 1253
    }
    if (
        type(year) is int and 
        year >= city_foundations[city] and
        year <= date.today().year
    ):
        return True
    else:
        print("The year should be integer and between {0} and {1}".format(city_foundations[city], date.today().year))
        return False
    

In [38]:
def input_to_dict(city, district, radius, floor, rooms, sq, year):
    
    if (check_city_district_radius_floor_rooms(city, district, radius, floor, rooms)
        and 
        check_sq(sq)
        and 
        check_year(year, city)
       ):
        X_check = pd.DataFrame({
            'city': city,
            'district': district,
            'floor': floor, 
            'rooms': rooms, 
            'sq': sq, 
            'year': year,
            'radius': radius
        }, index=[0])
        return X_check
    else:
        return None

In [39]:
# input your data

# X_check = input_to_dict(city='Warszawa', district='Śródmieście', radius=2, floor=3, rooms=2, sq=40, year=2000)
# X_check

,city,district,floor,rooms,sq,year,radius
0,Warszawa,Śródmieście,3,2,40,2000,2


In [4]:
def predict_by_input(X_check, cat_features, st_scaler, labels_dict, model):
    
    for col in cat_features:
        X_check[col] = X_check[col].apply(lambda x: labels_dict[col][x])

    X_check_scaled = st_scaler.transform(X_check)
    y_check_pred_model = model.predict(X_check_scaled)
    
    score = model.score(X_test_scaled, y_test)
    price_pred = np.round(y_check_pred_model[0])
    print("With a probability of {0}%, the prise will be about {1:,.0f} PLN ".format(
        round(score * 100, 1),
        round(price_pred)
    ))
    
    return price_pred, score

In [462]:
best_model = training_loop(experiment, model_class, grids[reg], X_train_scaled, y_train, X_test_scaled, y_test)['Run 1']['model']
predict_by_input(X_check, cat_features, st_scaler, labels_dict, best_model)


With a probability of 87.4%, the prise will be about 631,622 PLN 


In [40]:
71.5 / 2.5

28.6

In [41]:
82.5 / 2.5

33.0